<a href="https://colab.research.google.com/github/karthickrajas/Econometrics/blob/master/Portfolio_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# https://blog.quantinsti.com/portfolio-management-strategy-python/
# https://towardsdatascience.com/automating-portfolio-optimization-using-python-9f344b9380b9
# https://www.machinelearningplus.com/machine-learning/portfolio-optimization-python-example/
# https://evgenypogorelov.com/portfolio-rebalancing-python.html

!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install PyPortfolioOpt

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
from matplotlib.ticker import FuncFormatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "AAPL MSFT AMZN GOOGL META TSLA NVDA BRK-B JNJ UNH",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        # interval = "1m",

        # Whether to ignore timezone when aligning ticker data from 
        # different timezones. Default is True. False may be useful for 
        # minute/hourly data.
        ignore_tz = False,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

data = data.loc['2008-01-01':,:]
data.index = pd.to_datetime(data.index.date)

[*********************100%***********************]  10 of 10 completed


In [15]:
ls_stocks = "AAPL MSFT AMZN GOOGL META TSLA NVDA BRK-B JNJ UNH".split(' ')
",".join(ls_stocks)

'AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,BRK-B,JNJ,UNH'

In [5]:
cls_price = data.loc[:,data.columns.get_level_values(1)=='Close']
cls_price.columns = cls_price.columns.get_level_values(0)
cls_price 

,BRK-B,NVDA,MSFT,UNH,AAPL,TSLA,AMZN,GOOGL,META,JNJ
2008-01-02,92.099998,7.575688,25.928680,46.735233,5.941450,NaN,4.812500,17.146896,NaN,42.707218
2008-01-03,93.000000,7.516020,26.039108,46.949669,5.944195,NaN,4.760500,17.150400,NaN,42.720173
2008-01-04,91.000000,6.884902,25.310280,46.182701,5.490443,NaN,4.439500,16.441441,NaN,42.661842
2008-01-07,88.760002,6.173463,25.479614,46.908443,5.416955,NaN,4.441000,16.247498,NaN,43.322773
2008-01-08,88.300003,6.304276,24.625624,46.355892,5.222098,NaN,4.394000,15.807808,NaN,43.374607
...,...,...,...,...,...,...,...,...,...,...
2022-10-19,278.339996,120.510002,236.479996,520.320007,143.860001,222.039993,115.070000,99.629997,133.229996,164.690002
2022-10-20,273.529999,121.940002,236.149994,520.880005,143.389999,207.279999,115.250000,99.970001,131.529999,165.110001
2022-10-21,282.510010,124.660004,242.119995,533.729980,147.270004,214.440002,119.320000,101.129997,130.009995,168.710007
2022-10-24,287.480011,125.989998,247.250000,541.599976,149.449997,211.250000,119.820000,102.519997,129.720001,170.979996


In [19]:
def _get_indicator_data(data):
    """
    Function that uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for ticker in data.columns:

      # Also calculate moving averages for features
      data['ema50'+ticker] = data[ticker] / data[ticker].ewm(50).mean()
      data['ema21'+ticker] = data[ticker] / data[ticker].ewm(21).mean()
      data['ema15'+ticker] = data[ticker] / data[ticker].ewm(14).mean()
      data['ema5'+ticker] = data[ticker] / data[ticker].ewm(5).mean()
    
    return data

In [21]:
_get_indicator_data(cls_price)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,BRK-B,NVDA,MSFT,UNH,AAPL,TSLA,AMZN,GOOGL,META,JNJ,...,ema15GOOGL,ema5GOOGL,ema50META,ema21META,ema15META,ema5META,ema50JNJ,ema21JNJ,ema15JNJ,ema5JNJ
2008-01-02,92.099998,7.575688,25.928680,46.735233,5.941450,NaN,4.812500,17.146896,NaN,42.707218,...,1.000000,1.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
2008-01-03,93.000000,7.516020,26.039108,46.949669,5.944195,NaN,4.760500,17.150400,NaN,42.720173,...,1.000099,1.000093,NaN,NaN,NaN,NaN,1.000150,1.000148,1.000146,1.000138
2008-01-04,91.000000,6.884902,25.310280,46.182701,5.490443,NaN,4.439500,16.441441,NaN,42.661842,...,0.973067,0.974656,NaN,NaN,NaN,NaN,0.999197,0.999207,0.999215,0.999258
2008-01-07,88.760002,6.173463,25.479614,46.908443,5.416955,NaN,4.441000,16.247498,NaN,43.322773,...,0.971909,0.974718,NaN,NaN,NaN,NaN,1.010857,1.010713,1.010590,1.009947
2008-01-08,88.300003,6.304276,24.625624,46.355892,5.222098,NaN,4.394000,15.807808,NaN,43.374607,...,0.957508,0.962191,NaN,NaN,NaN,NaN,1.009533,1.009287,1.009079,1.008022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-19,278.339996,120.510002,236.479996,520.320007,143.860001,222.039993,115.070000,99.629997,133.229996,164.690002,...,0.982633,1.004048,0.811525,0.916060,0.949373,1.000381,0.981790,0.996159,1.000274,1.001549
2022-10-20,273.529999,121.940002,236.149994,520.880005,143.389999,207.279999,115.250000,99.970001,131.529999,165.110001,...,0.986909,1.006221,0.804306,0.908320,0.941196,0.989659,0.984597,0.998758,1.002636,1.003417
2022-10-21,282.510010,124.660004,242.119995,533.729980,147.270004,214.440002,119.320000,101.129997,130.009995,168.710007,...,0.998469,1.014870,0.798219,0.902012,0.934661,0.981785,1.005945,1.019583,1.022827,1.020990
2022-10-24,287.480011,125.989998,247.250000,541.599976,149.449997,211.250000,119.820000,102.519997,129.720001,170.979996,...,1.011371,1.023901,0.799630,0.904110,0.936787,0.982938,1.019091,1.031740,1.034066,1.028773


In [6]:
mu = expected_returns.mean_historical_return(cls_price)
cov = risk_models.sample_cov(cls_price)
ef = EfficientFrontier(mu, cov)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 29.8%
Annual volatility: 28.2%
Sharpe Ratio: 0.99


(0.29787363674545125, 0.2818897281965884, 0.9857529698693512)

In [7]:
log_returns = cls_price.pct_change().apply(lambda x: np.log(1+x))
cov_matrix = log_returns.cov()
corr_matrix = log_returns.corr()


In [8]:
cls_price.columns

Index(['BRK-B', 'NVDA', 'MSFT', 'UNH', 'AAPL', 'TSLA', 'AMZN', 'GOOGL', 'META',
       'JNJ'],
      dtype='object')

In [9]:
w = {i:weights[i] for i in weights}
port_var = cov_matrix.mul(w, axis=0).mul(w, axis=1).sum().sum()
(port_var*250)**0.5

0.2807003393814701

In [10]:
rebalancing_dates = data.resample('M').apply(lambda x: x[-1]).index
rebalancing_dates[0]

Timestamp('2008-01-31 00:00:00', freq='M')

In [ ]:
data.loc['2008-01-31'].pct_change()
np.log(data.loc['2008-01-31']) - np.log(data.loc['2008-01-31'].shift(1))

TSLA   Open            NaN
       High            NaN
       Low             NaN
       Close           NaN
       Volume          NaN
AMZN   Open            NaN
       High       0.135000
       Low       -0.136016
       Close      0.121070
       Volume    19.212663
NVDA   Open     -18.837281
       High       0.016503
       Low       -0.049929
       Close      0.031395
       Volume    16.160382
UNH    Open     -14.170180
       High       0.020754
       Low       -0.035846
       Close      0.030354
       Volume    12.203259
JNJ    Open     -12.248245
       High       0.024432
       Low       -0.026698
       Close      0.022746
       Volume    13.161574
MSFT   Open     -13.716350
       High       0.025678
       Low       -0.031650
       Close      0.027365
       Volume    15.278407
GOOGL  Open     -15.854600
       High       0.061152
       Low       -0.069947
       Close      0.054647
       Volume    17.555116
BRK-B  Open     -15.657403
       High       0.000000
 

In [11]:
def get_closing_values(data :pd.DataFrame, 
                       date: str
                       ) -> pd.DataFrame:
                       limited_data = data.loc[ date,:]
                       limited_data = limited_data.loc[limited_data.index.get_level_values(1)=='Close',:]
                       limited_data.index = [i[0] for i in limited_data.index]
                       return pd.DataFrame(limited_data.loc[ls_stocks])

In [ ]:
ls_stocks
port_wt = {i: round(1/(1+len(ls_stocks)),3) for i in ls_stocks}
port_wt = pd.DataFrame({i:[j] for (i,j) in port_wt.items()}).T 

In [ ]:
port_wt.T 

,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,BRK-B,JNJ,UNH
0,0.091,0.091,0.091,0.091,0.091,0.091,0.091,0.091,0.091,0.091


In [ ]:
check = get_closing_values(data, '2016-01-21')
check.T.reset_index(drop=True)

,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,BRK-B,JNJ,UNH
0,22.076778,45.385151,28.750999,36.3335,94.160004,13.331333,6.789661,125.620003,80.192268,102.458839


In [ ]:
tot_value = port_wt.T   * check.T.reset_index(drop=True)

In [16]:
tot_value

NameError: ignored